*Cell0 from my template*

In [1]:
import sys
from pathlib import Path

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = Path(this_path).parent.as_posix()
    else:
        newp = Path(this_path).as_posix()
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Markdown #, HTML
##display(HTML("<style>.container { width:100% !important; }</style>"))
#def new_section(title):
#    style = "text-align:center;background:#c2d3ef;padding:20px;color:#ffffff;font-size:3em;width:98%"
#    return HTML('<div style="{}">{}</div>'.format(style, title))


# Filtered dir() for method discovery:
def filter_module_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work together
              to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy down two levels only."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))

# these no longer work in JupyterLab
%load_ext autoreload
%autoreload 2 

Python ver: 3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]
Python env: dsml
Currrent dir: C:\Users\catch\Documents\GitHub\AI\Exercises\1_Constraint Satisfaction

Path added to sys.path: C:/Users/catch/Documents/GitHub/AI/Exercises/1_Constraint Satisfaction


---
# Constraint Satisfaction Problems
---
Constraint satisfaction is a general problem solving technique for solving a class of combinatorial optimization problems by imposing limits on the values in the solution. The goal of this exercise is to practice formulating some classical example problems as constraint satisfaction problems (CSPs), and then to explore using a powerful open source constraint satisfaction tool called [Z3](https://github.com/Z3Prover/z3) from Microsoft Research to solve them. Practicing with these simple problems will help you to recognize real-world problems that can be posed as CSPs; some solvers even have specialized utilities for specific types of problem (vehicle routing, planning, scheduling, etc.).

There are many different kinds of CSP solvers available for CSPs. Z3 is a "Satisfiability Modulo Theories" (SMT) solver, which means that unlike the backtracking and variable assignment heuristics discussed in lecture, Z3 first converts CSPs to satisfiability problems then uses a [boolean satisfiability](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem) (SAT) solver to determine feasibility. Z3 includes a number of efficient solver algorithms primarily developed to perform formal program verification, but it can also be used on general CSPs. Google's [OR tools](https://developers.google.com/optimization/) includes a CSP solver using backtracking with specialized subroutines for some common CP domains.

## I. The Road Ahead

0. [Cryptarithmetic](#I.-Cryptarithmetic) - introducing the Z3 API with simple word puzzles
0. [Map Coloring](#II.-Map-Coloring) - solving the map coloring problem from lectures
0. [N-Queens](#III.-N-Queens) - experimenting with problems that scale
0. [Revisiting Sudoku](#IV.-Revisiting-Sudoku) - revisit the sudoku project with the Z3 solver

<div class="alert alert-box alert-info">
NOTE: You can find solutions to this exercise in the "solutions" branch of the git repo, or on GitHub 
    <a href=https://github.com/udacity/artificial-intelligence/blob/solutions/Exercises/1_Constraint%20Satisfaction/AIND-Constraint_Satisfaction.ipynb> here.</a>
</div>

### Notes from a pythonista:  
<strong> I import z3 the pythonic way:</strong> 
```import z3``` 
instead of  
```from z3 import *``` as seen in numerous tutorials.  

<strong> Why?</strong>
<strong>
1.  It is not recommended as this can corrupt the namespace (some functions from different libraries could have the same name).
2.  I usually inspect a module **in order to learn** about its various functions and methods, so I used the root object for that (anlong with my function `filter_module_dir`).

The slight drawback &mdash; apart from from having to type a little more &mdash; is that some variables such as those returned by the `Solver.check()` function (i.e. `sat, unsat, unkown`), are not defined.  
The following lines use the [`CheckSatResult()` docs](https://z3prover.github.io/api/html/classz3py_1_1_check_sat_result.html) to define them:
</strong>

In [3]:
import z3

sat = z3.z3.CheckSatResult(z3.Z3_L_TRUE)
unsat = z3.z3.CheckSatResult(z3.Z3_L_FALSE)
# else: unkown

In [15]:
# from util import displayBoard
from collections import defaultdict
from itertools import product

---
# What's in Z3?

In [244]:
z3_mdl = filter_module_dir(z3)
z3_data = defaultdict(list)
for entry in z3_mdl:
    z3_data[entry[0]].append(entry)

z3_data['a']
z3_data['A']
z3_constants = z3_data['Z']
z3_constants[:10]

['addressof', 'alignment', 'append_log', 'args2params']

['ARRAY',
 'AlgebraicNumRef',
 'And',
 'AndThen',
 'ApplyResult',
 'ApplyResultObj',
 'ArgumentError',
 'ArithRef',
 'ArithSortRef',
 'Array',
 'ArrayRef',
 'ArraySort',
 'ArraySortRef',
 'Ast',
 'AstMap',
 'AstMapObj',
 'AstRef',
 'AstVector',
 'AstVectorObj',
 'AtLeast',
 'AtMost']

['Z3Exception',
 'Z3PPObject',
 'Z3_APP_AST',
 'Z3_ARRAY_SORT',
 'Z3_BOOL_SORT',
 'Z3_BV_SORT',
 'Z3_DATATYPE_SORT',
 'Z3_DEBUG',
 'Z3_DEC_REF_ERROR',
 'Z3_EXCEPTION']

In [245]:
z3.args2params?

Signature: z3.args2params(arguments, keywords, ctx=None)
Docstring:
Convert python arguments into a Z3_params object.
A ':' is added to the keywords, and '_' is replaced with '-'

>>> args2params(['model', True, 'relevancy', 2], {'elim_and' : True})
(params model true relevancy 2 elim_and true)
File:      c:\users\catch\anaconda3\envs\dsml\lib\site-packages\z3\z3.py
Type:      function


In [250]:
x, y = z3.Ints('x y')
s = z3.Solver()
s.add((x % 4) + 3 * (y / 2) > x - y)
print(s.sexpr())

s.check()
m = s.model()
m

(declare-fun y () Int)
(declare-fun x () Int)
(assert (> (+ (mod x 4) (* 3 (div y 2))) (- x y)))



sat

[y = 1, x = 1]

In [252]:
m.decls()

[y, x]

---
## I. Cryptarithmetic

We'll start by exploring the Z3 module with a _very_ simple & classic CSP problem called cryptarithmetic.  
A cryptarithmetic puzzle is posed as an arithmetic equation made up of words where each letter represents a distinct digit in the range (0-9). (This problem has no practical significance in AI, but it is a useful illustration of the basic ideas of CSPs.)  
For example, consider the problem and one possible solution shown below:

```
  T W O  :    9 3 8
+ T W O  :  + 9 3 8
-------  :  -------
F O U R  :  1 8 7 6
```
There are six distinct variables (F, O, R, T, U, W), and we have the following constraints on each letter:  
1. Each letter represents a disctinct number (e.g., F != O, R != T, ..., etc.) 
2. No leading zeros (i.e., T != 0 and F != 0)  
Then one possible solution is (F=1, O=8, R=6, T=9, U=7, W=3). 

### IMPLEMENTATION: Declaring Variables
For this problem we need a single variable for each distinct letter in the puzzle, and each variable will have an integer values between 0-9. (We will handle restricting the leading digits separately.)  
Complete the declarations in the next cell to create all of the remaining variables and constraint them to the range 0-9.

### IMPLEMENTATION: Encoding Assumptions as Constraints
We had two additional assumptions that need to be added as constraints: 1) leading digits cannot be zero, and 2) no two distinct letters represent the same digits. The first assumption can simply be added as a boolean statement like M != 0. And the second is a _very_ common CSP constraint (so common, in fact, that most libraries have a built in function to support it); z3 is no exception, with the Distinct(var_list) constraint function.

In [67]:
ca_solver = z3.Solver()  # create an instance of a Z3 CSP solver

F, O, R, T, U, W = z3.Ints('F O R T U W')  # create variables instance

# each cell contains a decimal digit (a value in {1, ..., 9}):
c_zero_not_first = [ z3.And(1<=D, D<=9) for D in z3.Ints('F T') ]
c_digit = [ z3.And(0<=D, D<=9) for D in z3.Ints('O R U W') ]
c_distinct = [ z3.Distinct([F, O, R, T, U, W]) ]

c_input = c_digit + c_zero_not_first + c_distinct

ca_solver.add(c_input)

Let's check the logic so far:

In [68]:
ca_solver.check()

sat

### Choosing Problem Constraints
There are often multiple ways to express the constraints for a problem.  
For example, in this case we could write a single large constraint combining all of the letters simultaneously  
$T\times10^2 + W\times10^1 + O\times10^0 + T\times10^2 + W\times10^1 + O\times10^0 = F\times10^3 + O\times10^2 + U\times10^1 + R\times10^0$.  

This kind of constraint works fine for some problems, but large constraints cannot usually be evaluated for satisfiability unless every variable is bound to a specific value.  
Expressing the problem with smaller constraints can sometimes allow the solver to finish faster.  

For example, we can break out each pair of digits in the summands and introduce a carry variable for each column:  
$(O + O)\times10^0 = R\times10^0 + carry_1\times10^1$  

This constraint can be evaluated as True/False with only four values assigned.  
The choice of encoding on this problem is unlikely to have any effect (because the problem is so small), however it is worth considering on more complex problems.

### Implementation: Add the Problem Constraints
Pick one of the possible encodings discussed above and add the required constraints into the solver in the next cell. 

In [272]:
def ca_solver_init():
    # create a Z3 CSP solver instance
    ca_solver = z3.Solver()
    # create variables instance
    #F, O, R, T, U, W = z3.Ints('F O R T U W')  
    # each F, T vars contains a decimal digit (a value in {1, ..., 9}):
    c_zero_not_first = [ z3.And(1<=D, D<=9) for D in z3.Ints('F T') ]
    # the others can be 0
    c_digit = [ z3.And(0<=D, D<=9) for D in z3.Ints('O R U W') ]
    # all must be different
    c_distinct = [ z3.Distinct([F, O, R, T, U, W]) ]

    c_input = c_digit + c_zero_not_first + c_distinct
    
    # add all input constraints to the solver:
    ca_solver.add(c_input)
    return ca_solver


def ca_solver_results(s):
    if s.check() == sat:
        out = {}
        m = s.model()
        out = dict((str(L), m.evaluate(L)) for L in [F,O,U,R,T,W])

        print(f"  T W O  :    {out['T']} {out['W']} {out['O']}")
        print(f"+ T W O  :  + {out['T']} {out['W']} {out['O']}")
        print("-------  :  -------")
        print(f"F O U R  :  {out['F']} {out['O']} {out['U']} {out['R']}")
    else:
        print("failed to solve")

## 1. Primary solution using single constraint for the cryptarithmetic equation

In [276]:
ca_solver = ca_solver_init()
ca_solver.check()

c_problem1 = [ 2*T*100 + 2*W*10 + 2*O == F*1000 + O*100 + U*10 + O ]
ca_solver.add(c_problem1)

ca_solver_results(ca_solver)

ca_solver.model().decls()

sat

  T W O  :    5 3 0
+ T W O  :  + 5 3 0
-------  :  -------
F O U R  :  1 0 6 4


[R, O, W, U, F, T]

## 2. Alternate solution using constraint with carry variables:

* $(2*O)\times10^0 = R\times10^0 + carry_1\times10^1$ 
* $(2*W)\times10^1 = U\times10^1 + carry_2\times10^2$  
* $(2*T)\times10^2 = O\times10^2 + carry_3\times10^3$  
* $F  = carry_3$ 

In [274]:
ca_solver = ca_solver_init()

# Add carry variables:
c10, c100, c1000 = z3.Ints('c10 c100 c1000')

# extra variables have conditions: not necessary (bc Ints?)
#c_carry = [ z3.And(c >= 0, c <= 9) for c in [c10, c100, c1000] ]
#ca_solver.add(c_carry)

ca_solver.add(2*O == R + 10 * c10)
ca_solver.add(2*W + c10 == U + 10 * c100)
ca_solver.add(2*T + c100 == O + 10 * c1000)
ca_solver.add(F == c1000)

ca_solver_results(ca_solver)

  T W O  :    8 4 6
+ T W O  :  + 8 4 6
-------  :  -------
F O U R  :  1 6 9 2


In [243]:
ca_solver?

Type:        Solver
String form:
[And(O >= 0, O <= 9),
           And(R >= 0, R <= 9),
           And(U >= 0, U <= 9),
           And(W >= 0, W <= 9),
           And(F >= <...> >= 1, T <= 9),
           Distinct(F, O, R, T, U, W),
           2*T*100 + 2*W*10 + 2*O == F*1000 + O*100 + U*10 + O]
File:        c:\users\catch\anaconda3\envs\dsml\lib\site-packages\z3\z3.py
Docstring:   Solver API provides methods for implementing the main SMT 2.0 commands: push, pop, check, get-model, etc.


### Cryptarithmetic Challenges
0. Search online for [more cryptarithmetic puzzles](https://www.reddit.com/r/dailyprogrammer/comments/7p5p2o/20180108_challenge_346_easy_cryptarithmetic_solver/) (or create your own). Come to office hours or join a discussion channel to chat with your peers about the trade-offs between monolithic constraints & splitting up the constraints. (Is one way or another easier to generalize or scale with new problems? Is one of them faster for large or small problems?)
0. Can you extend the solution to handle complex puzzles (e.g., using multiplication WORD1 x WORD2 = OUTPUT)?

# [Reddit] Challenge #346 [Easy] Cryptarithmetic Solver 

## Challenge Input:    
```
"WHAT + WAS + THY == CAUSE"

"HIS + HORSE + IS == SLAIN"

"HERE + SHE == COMES"

"FOR + LACK + OF == TREAD"

"I + WILL + PAY + THE == THEFT"
```

## Bonus:
```
"TEN + HERONS + REST + NEAR + NORTH + SEA + SHORE + AS + TAN + TERNS + SOAR + TO + ENTER + THERE + AS + HERONS + NEST + ON + STONES + AT + SHORE + THREE + STARS + ARE + SEEN + TERN + SNORES + ARE + NEAR == SEVVOTH"

"SO + MANY + MORE + MEN + SEEM + TO + SAY + THAT + THEY + MAY + SOON + TRY + TO + STAY + AT + HOME +  SO + AS + TO + SEE + OR + HEAR + THE + SAME + ONE + MAN + TRY + TO + MEET + THE + TEAM + ON + THE + MOON + AS + HE + HAS + AT + THE + OTHER + TEN == TESTS"

"THIS + A + FIRE + THEREFORE + FOR + ALL + HISTORIES + I + TELL + A + TALE + THAT + FALSIFIES + ITS + TITLE + TIS + A + LIE + THE + TALE + OF + THE + LAST + FIRE + HORSES + LATE + AFTER + THE + FIRST + FATHERS + FORESEE + THE + HORRORS + THE + LAST + FREE + TROLL + TERRIFIES + THE + HORSES + OF + FIRE + THE + TROLL + RESTS + AT + THE + HOLE + OF + LOSSES + IT + IS + THERE + THAT + SHE + STORES + ROLES + OF + LEATHERS + AFTER + SHE + SATISFIES + HER + HATE + OFF + THOSE + FEARS + A + TASTE + RISES + AS + SHE + HEARS + THE + LEAST + FAR + HORSE + THOSE + FAST + HORSES + THAT + FIRST + HEAR + THE + TROLL + FLEE + OFF + TO + THE + FOREST + THE + HORSES + THAT + ALERTS + RAISE + THE + STARES + OF + THE + OTHERS + AS + THE + TROLL + ASSAILS + AT + THE + TOTAL + SHIFT + HER + TEETH + TEAR + HOOF + OFF + TORSO + AS + THE + LAST + HORSE + FORFEITS + ITS + LIFE + THE + FIRST + FATHERS + HEAR + OF + THE + HORRORS + THEIR + FEARS + THAT + THE + FIRES + FOR + THEIR + FEASTS + ARREST + AS + THE + FIRST + FATHERS + RESETTLE + THE + LAST + OF + THE + FIRE + HORSES + THE + LAST + TROLL + HARASSES + THE + FOREST + HEART + FREE + AT + LAST + OF + THE + LAST + TROLL + ALL + OFFER + THEIR + FIRE + HEAT + TO + THE + ASSISTERS + FAR + OFF + THE + TROLL + FASTS + ITS + LIFE + SHORTER + AS + STARS + RISE + THE + HORSES + REST + SAFE + AFTER + ALL + SHARE + HOT + FISH + AS + THEIR + AFFILIATES + TAILOR + A + ROOFS + FOR + THEIR + SAFE == FORTRESSES"
```

In [271]:
class CryptoArithmetic():
    def __init__(self, crypto, op='+', solution=None):
        msg = 'Crypto format: "WORD + IS + THIS == MUCH"\n'
        if op not in crypto:
            raise TypeError(msg+f'Missing operator {op}')
        if '==' not in crypto:
            raise TypeError(msg+f'Missing equal "=="')
            
        self.op = op.strip()   
        self.n_plus = crypto.count(op)
        
        words = [w.strip() for w in crypto.split(op)]
        end = [l.strip() for l in words.pop(-1).split('==')]
        words.append(end[0])
        words.append(end[1])
        letters = set([''.join(l) for w in words for l in w])
        words_len = [len(w) for w in words]
        
        self.sum_word = end[1]
        self.words = words
        self.max_len = max(words_len)
        self.letters = letters
        self.initial_lines = self._get_initial_lines(crypto)
        self.solution = solution
        self.solution_lines = self._get_solution_lines()
    
    def _get_initial_lines(self, crypto):
        """Pretty print the cryptarithmetric sting and its solution.
        Arguments:
        ----------
        (str) crypto format: "WORD + IS + THIS == MUCH"
        (dict) solution: letter as key
        """
        lines = [] # -> [('T W O ', 'TWO')]
        line = ''
        for p in range(n_plus + 2):
            s = ''
            for l in words[p]:
                s += f' {l}'
                
            if p == (n_plus + 1):
                sub = '{0:{fill}>{width}}'.format('-',fill='-', width=1+self.max_len*2)
                lines.append((sub, '-'))
            if (p > 0) and (p < (n_plus + 1)):
                line = '{op} {0:>{width}}'.format(s, op=self.op,
                                                  width=self.max_len*2-2)
            else:
                line = '{:>{width}}'.format(s, width=self.max_len*2)
            lines.append((line, words[p]))
            
        return(lines)
    
    
    def _get_solution_lines(self):
        if self.solution is None:
            return []
        else:
            lines = []
            for i, (line, word) in enumerate(self.initial_lines):
                values = ''
                for l in word:
                    values += f' {solution[l]}'
                    
                if (i > 0) and (i < len(self.initial_lines)-1):
                    line += ' -> ' + '{op} {0:>{width}}'.format(values,
                                                                op=self.op,
                                                                width=self.max_len*2-2)
                else:
                    line += ' -> ' + '{:>{width}}'.format(values, width=self.max_len*2)
                lines.append(line)
            return lines
        
        
    def display(self):
        s = ''
        if self.solution is None:
            for L in self.initial_lines:
                s +=f'{L[0]}\n'
            print(s)
        else:
            pass

In [270]:
crypto_str = "THIS + IS + HIS == CLAIM"

crypto = CryptoArithmetic(crypto_str)
crypto.display()

TypeError: append() takes exactly one argument (2 given)

---
## II. Map Coloring

[Map coloring](https://en.wikipedia.org/wiki/Map_coloring) is a classic example of CSPs. A map coloring problem is specified by a set of colors and a map showing the borders between distinct regions. A solution to a map coloring problem is an assignment of one color to each region of the map such that no pair of adjacent regions have the same color.

Run the first cell below to declare the color palette and a solver. The color palette specifies a mapping from integer to color. We'll use integers to represent the values in each constraint; then we can decode the solution from Z3 to determine the color applied to each region in the map.

![Map coloring is a classic example CSP](map.png)

In [ ]:
# create instance of Z3 solver & declare color palette
mc_solver = Solver()
colors = {'0': "Blue", '1': "Red", '2': "Green"}

### IMPLEMENTATION: Add Variables
Add a variable to represent each region on the map above. Use the abbreviated name for the regions: WA=Western Australia, SA=Southern Australia, NT=Northern Territory, Q=Queensland, NSW=New South Wales, V=Victoria, T=Tasmania. Add constraints to each variable to restrict it to one of the available colors: 0=Blue, 1=Red, 2=Green.

In [ ]:
WA = Int('WA')
mc_solver.add(0 <= WA, WA <= 2)
# ...
# TODO: add the remaining six regions and color constraints

### IMPLEMENTATION: Distinct Adjacent Colors Constraints
As in the previous example, there are many valid ways to add constraints that enforce assigning different colors to adjacent regions of the map. One way is to add boolean constraints for each pair of adjacent regions, e.g., WA != SA; WA != NT; etc.

Another way is to use so-called pseudo-boolean cardinality constraint, which is a constraint of the form $ \sum w_i l_i = k $. Constraints of this form can be created in Z3 using `PbEq(((booleanA, w_A), (booleanB, w_B), ...), k)`. Distinct neighbors can be written with k=0, and w_i = 1 for all values of i. (Note: Z3 also has `PbLe()` for $\sum w_i l_i <= k $ and `PbGe()` for $\sum w_i l_i >= k $)

Choose one of the encodings discussed above and add the required constraints to the solver in the next cell.

In [ ]:
# TODO: add constraints to require adjacent regions to take distinct colors

In [ ]:
assert mc_solver.check() == sat, "Uh oh. The solver failed to find a solution. Check your constraints."
print("WA={}".format(colors[mc_solver.model()[WA].as_string()]))
print("NT={}".format(colors[mc_solver.model()[NT].as_string()]))
print("SA={}".format(colors[mc_solver.model()[SA].as_string()]))
print("Q={}".format(colors[mc_solver.model()[Q].as_string()]))
print("NSW={}".format(colors[mc_solver.model()[NSW].as_string()]))
print("V={}".format(colors[mc_solver.model()[V].as_string()]))
print("T={}".format(colors[mc_solver.model()[T].as_string()]))

#### Map Coloring Challenge Problems
1. Generalize the procedure for this problem and try it on a larger map (countries in Africa, states in the USA, etc.)
2. Extend your procedure to perform [graph coloring](https://en.wikipedia.org/wiki/Graph_coloring) (maps are planar graphs; extending to all graphs generalizes the concept of "neighbors" to any pair of connected nodes). (Note: graph coloring is [NP-hard](https://en.wikipedia.org/wiki/Graph_coloring#Computational_complexity), so it may take a very long time to color large graphs.)

---
## III. N-Queens

In the next problem domain you'll solve the 8-queens puzzle, then use it to explore the complexity of solving CSPs. The 8-queens problem asks you to place 8 queens on a standard 8x8 chessboard such that none of the queens are in "check" (i.e., no two queens occupy the same row, column, or diagonal). The N-queens problem generalizes the puzzle to to any size square board.

![The 8-queens problem is another classic CSP example](EightQueens.gif)

There are many acceptable ways to represent the N-queens problem, but one convenient way is to recognize that one of the constraints (either the row or column constraint) can be enforced implicitly by the encoding.  If we represent a solution as an array with N elements, then each position in the array can represent a column of the board, and the value at each position can represent which row the queen is placed on.

In this encoding, we only need a constraint to make sure that no two queens occupy the same row, and one to make sure that no two queens occupy the same diagonal.

#### IMPLEMENTATION: N-Queens Solver
Complete the function below to take an integer N >= 5 and return a Z3 solver instance with appropriate constraints to solve the N-Queens problem. NOTE: it may take a few minutes for the solver to complete the suggested sizes below.

In [ ]:
def Abs(x):
    return If(x >= 0, x, -x)

def nqueens(N):
    # TODO: Finish this function!
    raise NotImplementedError

In [ ]:
import time
from itertools import chain

runtimes = []
solutions = []
sizes = [8, 16, 32, 64]

for N in sizes:
    nq_solver = nqueens(N)
    start = time.perf_counter()
    assert nq_solver.check(), "Uh oh...The solver failed to find a solution. Check your constraints."
    end = time.perf_counter()
    print("{}-queens: {}ms".format(N, (end-start) * 1000))
    runtimes.append((end - start) * 1000)
    solutions.append(nq_solver)

plt.plot(sizes, runtimes)

### Queen Problem Challenges
- Extend the loop to run several times and estimate the variance in the solver. How consistent is the solver timing between runs?
- Read the `displayBoard()` function in the `util.py` module and use it to show your N-queens solution.

---
## IV. Revisiting Sudoku
For the last CSP we'll revisit Sudoku from the first project. You previously solved Sudoku using backtracking search with constraint propagation. This time you'll re-write your solver using Z3. The backtracking search solver relied on domain-specific heuristics to select assignments during search, and to apply constraint propagation strategies (like elimination, only-choice, naked twins, etc.). The Z3 solver does not incorporate any domain-specific information, but makes up for that by incorporating a more sophisticated and a compiled solver routine.

![Example of an easy sudoku puzzle](sudoku.png)

In [ ]:
from itertools import chain  # flatten nested lists; chain(*[[a, b], [c, d], ...]) == [a, b, c, d, ...]
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = [[Int("{}{}".format(r, c)) for c in cols] for r in rows]  # declare variables for each box in the puzzle
s_solver = Solver()  # create a solver instance

#### IMPLEMENTATION: General Constraints
Add constraints for each of the following conditions:
- Boxes can only have values between 1-9 (inclusive)
- Each box in a row must have a distinct value
- Each box in a column must have a distinct value
- Each box in a 3x3 block must have a distinct value

In [ ]:
# TODO: Add constraints that every box has a value between 1-9 (inclusive)
s_solver.add( # YOUR CODE HERE )

# TODO: Add constraints that every box in a row has a distinct value
s_solver.add( # YOUR CODE HERE )

# TODO: Add constraints that every box in a column has a distinct value
s_solver.add( # YOUR CODE HERE )

# TODO: Add constraints so that every box in a 3x3 block has a distinct value
s_solver.add( # YOUR CODE HERE )

#### IMPLMENTATION: Puzzle-Specific Constraints
Given the hints provided in the initial puzzle layout, you must also add constraints binding the box values to the specified values. For example, to solve the example puzzle you must specify A3 == 3 and B1 == 9, etc. The cells with a value of zero in the board below are "blank", so you should **not** create any constraint with the associate box.

In [ ]:
# use the value 0 to indicate that a box does not have an assigned value
board = ((0, 0, 3, 0, 2, 0, 6, 0, 0),
         (9, 0, 0, 3, 0, 5, 0, 0, 1),
         (0, 0, 1, 8, 0, 6, 4, 0, 0),
         (0, 0, 8, 1, 0, 2, 9, 0, 0),
         (7, 0, 0, 0, 0, 0, 0, 0, 8),
         (0, 0, 6, 7, 0, 8, 2, 0, 0),
         (0, 0, 2, 6, 0, 9, 5, 0, 0),
         (8, 0, 0, 2, 0, 3, 0, 0, 9),
         (0, 0, 5, 0, 1, 0, 3, 0, 0))

# TODO: Add constraints boxes[i][j] == board[i][j] for each box where board[i][j] != 0
s_solver.add( # YOUR CODE HERE )

In [ ]:
assert s_solver.check() == sat, "Uh oh. The solver didn't find a solution. Check your constraints."
for row, _boxes in enumerate(boxes):
    if row and row % 3 == 0:
        print('-'*9+"|"+'-'*9+"|"+'-'*9)
    for col, box in enumerate(_boxes):
        if col and col % 3 == 0:
            print('|', end='')
        print(' {} '.format(s_solver.model()[box]), end='')
    print()

#### Sudoku Challenges
1. Solve the "[hardest sudoku puzzle](# https://www.telegraph.co.uk/news/science/science-news/9359579/Worlds-hardest-sudoku-can-you-crack-it.html)"
2. Search for "3d Sudoku rules", then extend your solver to handle 3d puzzles